In [ ]:
!pip install langchain langchain-openai langgraph pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 9.6 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

In [ ]:
ROLES = {
    "1": {
        "name": "일반 지식 전문가",
        "description": "폭넓은 분야의 일반적인 질문에 답변",
        "details": "폭넓은 분야의 일반적인 질문에 대해 정확하고 이해하기 쉬운 답변을 제공하세요."
    },
    "2": {
        "name": "생성형 AI 제품 전문가",
        "description": "생성형 AI와 관련된 제품, 기술에 관한 전문적인 질문에 답변",
        "details": "생성형 AI와 관련 제품, 기술에 관현 전문적인 질문에 대해 최신 정보와 깊은 통찰력을 제공하세요."
    },
    "3": {
        "name": "카운슬러",
        "description": "개인적인 고민이나 심리적인 문제에 대해 지원 제공",
        "details": "개인적인 고민이나 심리적인 문제에 대해 공감적이고 지원적인 답변을 제공하고, 가능하다면 적절한 조언도 해주세요."
    }
}

In [ ]:
# 스테이트 정의
import operator
from typing import Annotated
from pydantic import BaseModel, Field

class State(BaseModel):
    query: str = Field(..., description="사용자의 질문")
    current_role: str = Field(default="", description="선정된 답변 역할")
    messages: Annotated[list[str], operator.add] = Field(default=[], description="답변 기록")
    current_judge: bool = Field(default=False, description="품질 체크 결과")
    judgement_reason: str = Field(default="", description="품질 체크 판정 이유")

# Chat Model 초기화
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

llm = ChatOpenAI(model="gpt-5-nano", temperature=0.0)
# 나중에 max_tokens 값을 변경할 수 있도록 변경 가능한 필드 선언
# llm = llm.configurable_fields(max_tokens=ConfigurableField(id="max_tokens"))

In [ ]:
# Selection 노드 구현
from typing import Any
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def selection_node(state: State) -> dict[str, Any]:
  query = state.query
  role_options = "\n".join([f"{k}. {v['name']}: {v['description']}" for k, v in ROLES.items()])

  prompt = ChatPromptTemplate.from_template("""
  질문을 분석하고, 가장 적절한 답변 담당 역할을 선택하세요.
  선택지:
  {role_options}

  답변은 선택지의 번호(1, 2, 또는 3)만 반환하세요.

  질문: {query}
  """.strip())

  # 선택지의 번호만 반환하기를 기대하므로 max_tokens 값을 1로 변경
  chain = prompt | llm.with_config(configurable={"max_tokens": 1}) | StrOutputParser()

  role_number = chain.invoke({"role_options": role_options, "query": query})

  selected_role = ROLES[role_number.strip()]["name"]

  return {"current_role": selected_role}

In [ ]:
# Answering 노드 구현
def answering_node(state: State) -> dict[str, Any]:
  query = state.query
  role = state.current_role
  role_details = "\n".join([f"- {v['name']}: {v['details']}" for v in ROLES.values()])

  prompt = ChatPromptTemplate.from_template("""
  당신은 {role}로서 답변하세요. 다음 질문에 대해 당신의 역할에 기반한 적절한 답변을 제공하세요.

  역할 상세:
  {role_details}

  질문: {query}

  답변:
  """.strip())

  chain = prompt | llm | StrOutputParser()
  answer = chain.invoke({"role": role, "role_details": role_details, "query": query})
  return {"messages": [answer]}

In [ ]:
# Check 노드 구현
class Judgement(BaseModel):
  judge: bool = Field(default=False, description="판정 결과")
  reason: str = Field(default="", description="판정 이유")

def check_node(state: State) -> dict[str, Any]:
  query  = state.query
  answer = state.messages[-1]
  prompt = ChatPromptTemplate.from_template(
"""다음 답변의 품질을 체크하고, 문제가 있으면 'False', 문제가 없으면 'True'로 답변하세요.
또한, 그 판정 이유도 설명하세요.

사용자의 질문: {query}
답변: {answer}
""".strip()
  )

  chain = prompt | llm.with_structured_output(Judgement)

  result: Judgement = chain.invoke({"query": query, "answer": answer})

  return {
      "current_judge": result.judge,
      "judgement_reason": result.reason
  }

In [ ]:
# 그래프 생성
from langgraph.graph import StateGraph

workflow = StateGraph(State)

# 노드 추가
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_node)
workflow.add_node("check", check_node)

# 에지 정의
# selection 노드에서 처리 시작
workflow.set_entry_point("selection")
# selection 노드에서 answering 노드로
workflow.add_edge("selection", "answering")
# answering 노드에서 check 노드로
workflow.add_edge("answering", "check")

# 조건부 에지 정의
from langgraph.graph import END
workflow.add_conditional_edges(
    "check",
    lambda state: state.current_judge,
    {True: END, False: "selection"}
)

compiled = workflow.compile()

initial_state = State(query="생성형 AI에 관해 알려주세요.")
result = compiled.invoke(initial_state)

In [ ]:
import json

print(json.dumps(result, ensure_ascii=False, indent=2))

{
  "query": "생성형 AI에 관해 알려주세요.",
  "current_role": "생성형 AI 제품 전문가",
  "messages": [
    "다음은 생성형 AI에 대해 일반 지식, 제품 관점, 그리고 필요한 경우 개인 상담 관점까지 포괄적으로 정리한 답변입니다.\n\n1) 생성형 AI란 무엇인가\n- 정의: 데이터를 바탕으로 새로운 콘텐츠를 생성하는 AI 기술의 총칭으로, 텍스트, 이미지, 코드, 음악, 영상 등 다양한 형식의 창작물이 가능하도록 학습된 모델을 말합니다.\n- 핵심 기술: 대형 언어 모델(Large Language Models, LLM), 확산 모델(Diffusion Models) 등으로 구성되며, 멀티모달 모델은 텍스트-이미지-음성 등을 함께 다룹니다.\n- 학습 방식의 구성 요소:\n  - 사전 학습(pretraining): 방대한 데이터로 패턴과 구조를 학습.\n  - 미세조정(finetuning)/지시적 학습(instruction tuning): 특정 작업에 맞춘 성능 향상.\n  - 사람의 피드백을 활용한 학습(RLHF: Reinforcement Learning from Human Feedback): 모델의 출력 품질을 사람의 선호에 맞춰 정렬.\n  - 검색(RAG: Retrieval-Augmented Generation): 외부 지식 소스와 연결해 사실성 및 최신성 보강.\n- 적용 범위: 텍스트 생성, 코드 작성, 디자인 및 이미지 생성, 데이터 보강, 시뮬레이션, 교육 도구 등 광범위.\n\n2) 작동 원리의 간단한 그림\n- 입력 프롬프트(prompt) 또는 명령으로 시작.\n- 모델은 학습된 패턴과 맥락을 바탕으로 결과물을 생성.\n- 필요 시 외부 도구(계산기, 데이터베이스, 검색 엔진, 코드 런타임 등)와 연결해 기능 확장.\n- 출력물은 사람의 검토를 통해 수정-승인될 수 있으며, 피드백 루프를 통해 성능이 개선됩니다.\n\n3) 현재 트렌드와 대표 사례\n- 멀티모달isation: 텍스트뿐 아니라 

In [ ]:
# 그래프 구조 시각화
# apt-get install graphviz libgraphviz-dev pkg-config
# pip install pygraphviz

# from IPython.display import Image
# Image(compiled.get_graph().draw_png())